# lesson 4

**背景：**
真核生物的基因包括了外显子和内含子，但转录成RNA后，内含子往往会被切除，所有外显子的序列首尾相接，形成能够编码蛋白质的成熟mRNA序列。然而，一条完整的mRNA序列，并不是全部序列都能翻译成蛋白质，只有中间的CDS区(也叫Open reading frame、Protein coding region)才能翻译，CDS区两侧分别是5’UTR和3’UTR。真核基因的结构以及对应的mRNA结构，可以参考以下两个链接中的示意图：

https://en.wikipedia.org/wiki/Gene    ＃Structure and function部分的第一个图。思考题：5’UTR、3’UTR、CDS跟exon的关系是什么。

https://en.wikipedia.org/wiki/Messenger_RNA    ＃Structure部分的图。思考题：5’Cap和Poly-A tail是不是由基因组编码的。

而我们平常做基因注释，往往只能把每个基因的CDS区给注释出来(见之前给大家的GFF文件出现的大量CDS标签)。所以，只要把每个基因的CDS序列提取出来，然后首尾相接，即可用于翻译成蛋白质的序列。

编程任务：根据Camponotus floridanus蚂蚁的基因组fasta文件以及配套的基因注释gff文件，1)获取每个基因去除内含子后的CDS序列，以fasta格式输出；2)根据已知的密码子表，把CDS翻译成蛋白质序列，以fasta格式输出。

data: cm.fa.gz; cflo_v3.3.gff.gz

## answer

下图为真核生物蛋白编码基因结构图：

![eukaryotic protein-coding gene](https://upload.wikimedia.org/wikipedia/commons/5/54/Gene_structure_eukaryote_2_annotated.svg)

从上图看出，5'UTR和3'UTR会被转录，而后保留在成熟mRNA中，因此5'UTR和3'UTR属于exon部分，而CDS可以理解为Pre-mRNA经过剪接，去掉intron后将exon连接起来的产物，即Protein coding region。

下图为真核生物成熟mRNA结构图：

![mRNA_structure](https://upload.wikimedia.org/wikipedia/commons/b/ba/MRNA_structure.svg)

5'cap和3'poly-A tail不是转录得来，而是从头合成并连接到5'UTR和3'UTR上。

## 提取每个基因所有的CDS

In [ ]:
# lesson4 practice
# date: 2018/08/02
# Author: zxzhu

def get_CDS(gff, genome_sequence, out):   #获取cds信息，每个基因对应[scaffold, strand, (cds_start, cds_end)]。
    gene = {}                             #存储每个基因所有cds的信息。
    genome = {}                           #存储每个scaffold对应的序列信息。
    g = open(gff, 'r')
    for line in g:
        line = line.strip()
        if 'mRNA' in line:               
            line = line.split('\t')
            gene_name = line[-1].split(';')[0].split('=')[-1]
            gene[gene_name] = [line[0], line[6]]

        else:                                       
            line = line.split('\t')
            gene_name = line[-1].split(';')[0].split('=')[-1]
            if line[7] != '.':
                position = (int(line[3]) + int(line[7]), int(line[4]))
                gene[gene_name].append(position)
            else:
                position = (int(line[3]), int(line[4]))
                gene[gene_name].append(position)
    g.close()
    print('data read')
    
    f = open(genome_sequence, 'r')
    scaffold = ''
    tmp = []
    for line in f:
        line = line.strip()
        if line.startswith('>'):
            if tmp:
                genome[scaffold] = ''.join(tmp)
                scaffold = line[1:]
                tmp = []
            else:
                scaffold = line[1:]
        else:
            tmp.append(line)
    genome[scaffold] = ''.join(tmp)
    f.close()
    print('data read')
    
    get_fasta(gene,genome,out)

def get_fasta(gene, genome, out):
    w = open(out, 'w')
    for g, cds in gene.items():
        sequence = genome[cds[0]]     
        strand = cds[1]               
        position = sorted(cds[2:], key=lambda x: x[0])   
        if strand == '+':
            w.write('>{}\n'.format(g))
            cds_sequence = ''.join([sequence[start-1:end] for (start, end) in position])                  #按顺序拼接所有cds。
            w.write('\n'.join([cds_sequence[i:i+80] for i in range(0, len(cds_sequence), 80)]) + '\n')    #80个碱基一行。

        if strand == '-':
            w.write('>{}\n'.format(g))
            cds_sequence = ''.join([sequence[start-1:end] for (start, end) in position])
            cds_sequence = reverse(cds_sequence)         
            w.write('\n'.join([cds_sequence[i:i+80] for i in range(0, len(cds_sequence), 80)]) + '\n')
    w.close()
    print('done!')
    
def reverse(sequence):                           #反向互补。
    D = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}
    return ''.join([D[i] for i in sequence.upper()][::-1])

gff = './data/Traning/Lesson3/cflo_v3.3.gff'
genome_sequence = './data/Traning/Lesson2/cm.fa'
out = 'lesson4_cds.fa'
get_CDS(gff, genome_sequence, out)

运行以上脚本，可以得到lesson4_cds.fa文件，给出部分文件截图：

![mRNA](./image/lesson4.JPG)

需要注意的地方是：**每个gene对应多个cds,要按位置信息拼接好这些cds。如果是负链，应该进行反向互补。**

## 将CDS翻译为蛋白序列 

In [16]:
def codon2AA(codon,s):
    try:
        return codon[s]
    except KeyError:
        return 'X'

codon_file = './data/codon.txt'
f = open('./lesson4.fa', 'r')
tmp = []
gene = {}
for line in f:
    line = line.strip()
    if line.startswith('>'):
        if tmp:
            gene[cds] = ''.join(tmp)
            cds = line[1:]
            tmp = []
        else:
            cds = line[1:]
    else:
        tmp.append(line)
gene[cds] = ''.join(tmp)
f.close()
codon = {}
with open(codon_file, 'r') as f:
    for line in f:
        line = line.strip().split(' ')
        codon[line[0]] = line[-1]

w = open('lesson4_protein.fa', 'w')
for key, value in gene.items():
    w.write('>{}\n'.format(key))
    #print(key,len(value)/3)
    sequence = [value[i:i+3] for i in range(0, len(value), 3)]
    protein_sequence = ''.join([codon2AA(codon,s) for s in sequence])
    
    w.write('\n'.join([protein_sequence[i:i+80] for i in range(0, len(protein_sequence), 80)]) + '\n')
                   
w.close()

利用上述代码可以将CDS翻译为蛋白序列。核对后无误，截图展示：

![protein](./image/lesson4_protein.JPG)

## Reference

[GFF3](http://rice.bio.indiana.edu:7082/annot/gff3.html)

[Gene](https://en.wikipedia.org/wiki/Gene)

[Messenger_RNA](https://en.wikipedia.org/wiki/Messenger_RNA)